In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, count
from pyspark.sql import functions as F


In [2]:
# Criar a sessão Spark
spark = SparkSession.builder.appName("EnderecoClientesETL").getOrCreate()

In [3]:
# Lê o arquivo em formato parquet
df_enderecos_clientes = spark.read.parquet('C:\\Users\\Theuzao\\Desktop\\panvel_datalake\\data\\raw\\enderecos_clientes.parquet')

In [4]:
# Contar os valores faltantes em cada coluna
df_enderecos_clientes.select([count(when(col(c).isNull(), c)).alias(c) for c in df_enderecos_clientes.columns]).show()
# como o coluna d_dt_exc nao será usada para a solução do problema, nao será transformada

+--------+--------+--------+-----+----+
|v_id_cli|n_sq_end|d_dt_exc|v_lcl|v_uf|
+--------+--------+--------+-----+----+
|       0|       0|  133819|    0|   0|
+--------+--------+--------+-----+----+



In [5]:
df_enderecos_clientes.show() 

+--------------------+--------+--------------------+-----------------+----+
|            v_id_cli|n_sq_end|            d_dt_exc|            v_lcl|v_uf|
+--------------------+--------+--------------------+-----------------+----+
|00D3BFA61A4EB684B1F5| 7470026|                null|         CURITIBA|  PR|
|0313852B5434EC0A76D1| 6772409|                null|         LONDRINA|  PR|
|04194EF6A82A8F1C519C| 6459025|                null|SANTA CRUZ DO SUL|  RS|
|01AF764285A24153070B| 5912571|                null|     PORTO ALEGRE|  RS|
|04A52338FAB5D10204AD|       1|                null|         GRAVATAI|  RS|
|04D139F9FE86539E02F5|       1|                null|         CURITIBA|  PR|
|035A054182C4F2CFF563|       1|                null|         CURITIBA|  PR|
|0489FD8213030BE9A86B| 4404117|2023-03-17 10:42:...|     PORTO ALEGRE|  RS|
|04808AA2CCD1D61D82CB| 2614487|                null|     PORTO ALEGRE|  RS|
|025636ACDAC688833A0D|       1|                null|          PALHOCA|  SC|
|04164C0A709

In [6]:
# Converter todas as colunas para String após a leitura
df_enderecos_clientes = df_enderecos_clientes.select([F.col(col_name).cast("string").alias(col_name) for col_name in df_enderecos_clientes.columns])


In [7]:
df_enderecos_clientes.printSchema()

root
 |-- v_id_cli: string (nullable = true)
 |-- n_sq_end: string (nullable = true)
 |-- d_dt_exc: string (nullable = true)
 |-- v_lcl: string (nullable = true)
 |-- v_uf: string (nullable = true)



In [8]:
# Salvar o DataFrame em formato Parquet
df_enderecos_clientes.write.mode("overwrite").option("header", "true").csv("C:/Users/Theuzao/Desktop/panvel_datalake/data/transformed/endereco_clientes")


In [9]:
# Gravar os dados no PostgreSQL
# df_enderecos_clientes.write.jdbc(url=url, table="endereco_clientes", mode="overwrite", properties=properties)

# Encerra a sessão Spark
spark.stop()